In [20]:
from helpers.strings import get_obj_from_path
import pandas as pd
from tqdm.notebook import tqdm
import openpyxl

In [19]:
companies = Company.objects.all()

In [15]:
companies

<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>

In [16]:
def get_connected_reference(company, resource, key, value):
    select_options = get_obj_from_path(company.custom_options, "{}__fields__{}__selectoptions__options".format(resource, key))
    return next(a['name'] for a in select_options if a['value'] == value)

In [18]:
classes = []
faixas = []
sentidos = []
roads = []
status_list = []
equipes = []
for company in tqdm(companies):
    filename = 'CCRDados_'+company.name
    pd.DataFrame().to_excel(filename+'.xlsx')
    
    #Classes
    for form in OccurrenceType.objects.filter(company=company).order_by('occurrence_kind', 'name'):
        try:
            classes.append({'Natureza':get_connected_reference(company, "reporting", "occurrence_kind", form.occurrence_kind),
                'ID_natureza':form.occurrence_kind,
                'Classe':form.name,
                'UUID Classe':str(form.uuid)})
            print("|".join([
                get_connected_reference(company, "reporting", "occurrence_kind", form.occurrence_kind),
                form.occurrence_kind,
                form.name,
                str(form.uuid)
            ]))
        except StopIteration:
            continue
    df = pd.DataFrame(data=classes)
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df.to_excel(writer,sheet_name='Classes',na_rep="-")
    
    # Faixas
    for option in get_obj_from_path(company.custom_options, "reporting__fields__lane__selectoptions__options"):
        try:
            faixas.append({'Faixa': option['name'],
                'Id_Faixa':option['value']})
            print("|".join([
                option['name'],
                option['value'],
            ]))
        except StopIteration:
            continue
    df_faixa = pd.DataFrame(data=faixas)
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df_faixa.to_excel(writer,sheet_name="Faixas",na_rep="-")
        
    # Sentidos
    for option in get_obj_from_path(company.custom_options, "reporting__fields__direction__selectoptions__options"):
        try:
            sentidos.append({'Sentidos': option['name'],
                'Id_Sentidos':option['value']})
            print("|".join([
                option['name'],
                option['value'],
            ]))
        except StopIteration:
            continue
    df_sentido = pd.DataFrame(data=sentidos)
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df_sentido.to_excel(writer,sheet_name="Sentidos",na_rep="-")
        
    # Rodovias  
    for road in Road.objects.filter(company=company).order_by('name').values_list('name', flat=True).distinct():
        try:
            roads.append({'Rodovias':road})
            print(road)
        except StopIteration:
            continue
    df_road = pd.DataFrame(data=roads)
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df_road.to_excel(writer,sheet_name="Rodovias",na_rep="-")
        
        
    # Status
    for status in ServiceOrderActionStatus.objects.filter(companies=company).order_by('name'):
        try:
            status_list.append({'Status':status.name,
                'UUID_Status':str(status.uuid)})
            print("|".join([
                status.name,
                str(status.uuid),
            ]))
        except StopIteration:
            continue
    df_status = pd.DataFrame(data=status_list)
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df_status.to_excel(writer,sheet_name="Status",na_rep="-")
        
    # Equipes    
    for firm in Firm.objects.filter(company=company).order_by('name'):
        try:
            equipes.append({'Equipes': firm.name,
                'UUID_Equipe':str(firm.uuid)})
            print("|".join([
                firm.name,
                str(firm.uuid),
            ]))
        except StopIteration:
            continue
    df_equipes = pd.DataFrame(data=equipes)
    with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
        df_equipes.to_excel(writer,sheet_name="Equipes",na_rep="-")
        
    # deletar Sheet1
    workbook = openpyxl.load_workbook(filename+'.xlsx')
    del workbook['Sheet1']
    workbook.save(filename+'.xlsx')

  0%|          | 0/4 [00:00<?, ?it/s]

Levantamentos|1|Erro de marco quilométrico|64856e23-3b41-4d42-858a-14dbdefb4fe4
Inventário|2|Barreira New Jersey|d7153fa0-7a64-40d8-8a8f-ee473eb406b3
Inventário|2|Cadastro de Acessos|75000e27-c933-4f45-9e4f-f59809bf6b84
Inventário|2|Cadastro Inicial OAE|52b851f7-50df-42ad-acde-4e6cd046711b
Inventário|2|Defensas em OAE|30ff3296-86ed-4190-aa09-6a16b4ecf50e
Inventário|2|Defensas Metálicas|79ac53f8-f265-4375-b920-040b13457b80
Inventário|2|Drenagem Profunda Padrão CCR|5b386903-e7af-4c5c-aea2-a91299696115
Inventário|2|Drenagem Superficial|0b980e95-d542-48ac-901c-8aebfe7c02c1
Inventário|2|Edificações SGE|3f6988f9-1574-4843-b0b7-d31614bde620
Inventário|2|Equipamentos de ITS|61ef6f83-05db-4117-9328-4afa1c4cf812
Inventário|2|Iluminação|8dc21140-b8bb-49aa-b4de-a19a283325f0
Inventário|2|Instalações Elétricas de Edificações|96a909a1-8e54-4f1b-ac8c-7333bb597894
Inventário|2|Ocupações|c433fff6-b996-403b-ab4b-74638bdcd600
Inventário|2|Placas Pavimento de Concreto - ICP|8906ec62-b9d2-4442-91fb-a2278ccb

Faixa 1|1
Faixa 2|2
Faixa 3|3
Faixa 4|4
Acostamento|5
Acesso|51
Marginal 1|6
Marginal 2|7
Balança|8
Base Operacional|81
Viaduto/Ponte|9
Taper|10
Trevo|101
Túnel|102
Contorno|11
Retorno|12
Alça|13
Pista de serviço|141
Passarela|142
Passagem Inferior|14
Pedágio|143
Garrafão|15
Canteiro Central|16
Faixa Lateral|17
Bordo Direito|18
Bordo Esquerdo|19
Eixo|20
Dispositivo em nível|21
Faixa de domínio|22
Zebrado|23
Esquerda|24
Direita|25
Faixa de segurança|26
Interna|27
Externa|28
Expressa|29
Faixa 5|30
Faixa 6|31
Dispositivo|32
Marginal|33
Dispositivo Lateral|34
Leste|35
Marginal Leste|36
Marginal Norte|37
Marginal Sul|38
Norte|39
Oeste|40
Sul|41
Canteiro Lateral|42
Norte/Sul|45
Marginal oeste|46
Leste/Oeste|47
Via Marginal|48
Via Lateral|49
Não se aplica|99
Faixa Adicional|50
Faixa Adicional Crescente|52
Faixa Adicional Decrescente|53
Norte|0
Sul|1
Leste|2
Oeste|3
Norte/Sul|4
Leste/Oeste|5
Crescente|6
Decrescente|7
Canteiro Central|8
Ambos|9
Transversal|10
Não se Aplica|11
Externa|12
Interna

Faixa 1|1
Faixa 2|2
Faixa 3|3
Faixa 4|4
Acostamento|5
Acesso|51
Marginal 1|6
Marginal 2|7
Balança|8
Base Operacional|81
Viaduto/Ponte|9
Taper|10
Trevo|101
Túnel|102
Contorno|11
Retorno|12
Alça|13
Pista de serviço|141
Passarela|142
Passagem Inferior|14
Pedágio|143
Garrafão|15
Canteiro Central|16
Faixa Lateral|17
Bordo Direito|18
Bordo Esquerdo|19
Eixo|20
Dispositivo em nível|21
Faixa de domínio|22
Zebrado|23
Esquerda|24
Direita|25
Faixa de segurança|26
Interna|27
Externa|28
Expressa|29
Faixa 5|30
Faixa 6|31
Local|32
Central|33
Canteiro Lateral|34
Leste|35
Marginal Leste|36
Marginal Norte|37
Marginal Sul|38
Norte|39
Oeste|40
Sul|41
Marginal norte/sul|42
Interna/Externa|43
Norte/Sul|45
Marginal oeste|46
Leste/Oeste|47
Via Marginal|48
Via Lateral|49
Não se aplica|99
Faixa Adicional|50
Faixa Adicional Crescente|52
Faixa Adicional Decrescente|53
Norte|0
Sul|1
Leste|2
Oeste|3
Norte/Sul|4
Leste/Oeste|5
Crescente|6
Decrescente|7
Canteiro Central|8
Ambos|9
Transversal|10
Não se Aplica|11
Interna

Faixa 1|1
Faixa 2|2
Faixa 3|3
Faixa 4|4
Acostamento|5
Acesso|51
Marginal 1|6
Marginal 2|7
Balança|8
Base Operacional|81
Viaduto/Ponte|9
Taper|10
Trevo|101
Túnel|102
Contorno|11
Retorno|12
Alça|13
Pista de serviço|141
Passarela|142
Passagem Inferior|14
Pedágio|143
Garrafão|15
Canteiro Central|16
Faixa Lateral|17
Bordo Direito|18
Bordo Esquerdo|19
Eixo|20
Dispositivo em nível|21
Faixa de domínio|22
Zebrado|23
Esquerda|24
Direita|25
Faixa de segurança|26
Interna|27
Externa|28
Expressa|29
Faixa 5|30
Faixa 6|31
Norte/Sul|38
Marginal oeste|46
Leste/Oeste|47
Via Marginal|48
Via Lateral|49
Não se aplica|99
Faixa Adicional|50
Faixa Adicional Crescente|52
Faixa Adicional Decrescente|53
Norte|0
Sul|1
Leste|2
Oeste|3
Norte/Sul|4
Leste/Oeste|5
Crescente|6
Decrescente|7
Canteiro Central|8
Ambos|9
Transversal|10
Não se Aplica|11
Interna|12
Externa|13
Interna/Externa|14
Marginal oeste|15
Via Marginal|16
Via Lateral|17
BR-101 RJ
BR-101 SP
BR-116 RJ
BR-116 SP
Em contratação|fd32ea4d-5939-4586-96aa-5e6c5

Faixa 1|1
Faixa 2|2
Faixa 3|3
Faixa 4|4
Acostamento|5
Acesso|51
Marginal 1|6
Marginal 2|7
Balança|8
Base Operacional|81
Viaduto/Ponte|9
Taper|10
Trevo|101
Túnel|102
Contorno|11
Retorno|12
Alça|13
Pista de serviço|141
Passarela|142
Passagem Inferior|14
Pedágio|143
Garrafão|15
Canteiro Central|16
Faixa Lateral|17
Bordo Direito|18
Bordo Esquerdo|19
Eixo|20
Dispositivo em nível|21
Faixa de domínio|22
Zebrado|23
Esquerda|24
Direita|25
Faixa de segurança|26
Interna|27
Externa|28
Expressa|29
Faixa 5|30
Faixa 6|31
Marginal Interna|32
Marginal norte/sul|33
Marginal norte|34
Marginal sul|35
Interna/Externa|36
Canteiro Lateral|37
Norte/Sul|38
Marginal oeste|39
Leste/Oeste|40
Via Marginal|41
Via Lateral|42
Não se aplica|99
Faixa Adicional|43
Faixa Adicional Crescente|44
Faixa Adicional Decrescente|45
Norte|0
Sul|1
Leste|2
Oeste|3
Norte/Sul|4
Leste/Oeste|5
Crescente|6
Decrescente|7
Canteiro Central|8
Ambos|9
Transversal|10
Não se Aplica|11
Interna|12
Externa|13
Interna/Externa|14
Canteiro Lateral|1

In [7]:
# import pandas
# df = pandas.DataFrame(data=my_dict)
# df.to_excel('modera.xlsx')

In [8]:
for option in get_obj_from_path(company.custom_options, "reporting__fields__lane__selectoptions__options"):
    try:
        print("|".join([
            option['name'],
            option['value'],
        ]))
    except StopIteration:
        continue

Faixa 1|1
Faixa 2|2
Faixa 3|3
Faixa 4|4
Acostamento|5
Acesso|51
Marginal 1|6
Marginal 2|7
Balança|8
Base Operacional|81
Viaduto/Ponte|9
Taper|10
Trevo|101
Túnel|102
Contorno|11
Retorno|12
Alça|13
Pista de serviço|141
Passarela|142
Passagem Inferior|14
Pedágio|143
Garrafão|15
Canteiro Central|16
Faixa Lateral|17
Bordo Direito|18
Bordo Esquerdo|19
Eixo|20
Dispositivo em nível|21
Faixa de domínio|22
Zebrado|23
Esquerda|24
Direita|25
Faixa de segurança|26
Interna|27
Externa|28
Expressa|29
Faixa 5|30
Faixa 6|31
Norte/Sul|38
Marginal oeste|46
Leste/Oeste|47
Via Marginal|48
Via Lateral|49
Não se aplica|99
Faixa Adicional|50
Faixa Adicional Crescente|52
Faixa Adicional Decrescente|53


In [9]:
for option in get_obj_from_path(company.custom_options, "reporting__fields__direction__selectoptions__options"):
    try:
        print("|".join([
            option['name'],
            option['value'],
        ]))
    except StopIteration:
        continue

Norte|0
Sul|1
Leste|2
Oeste|3
Norte/Sul|4
Leste/Oeste|5
Crescente|6
Decrescente|7
Canteiro Central|8
Ambos|9
Transversal|10
Não se Aplica|11
Interna|12
Externa|13
Interna/Externa|14
Marginal oeste|15
Via Marginal|16
Via Lateral|17


In [10]:
for road in Road.objects.filter(company=company).order_by('name').values_list('name', flat=True).distinct():
    try:
        print(road)
    except StopIteration:
        continue

BR-101 RJ
BR-101 SP
BR-116 RJ
BR-116 SP


In [11]:
for status in ServiceOrderActionStatus.objects.filter(companies=company).order_by('name'):
    try:
        print("|".join([
            status.name,
            str(status.uuid),
        ]))
    except StopIteration:
        continue

Em contratação|fd32ea4d-5939-4586-96aa-5e6c5e1f5248
Em execução|173cc4b9-62d4-4cbc-8e6a-e12646d6dc20
Encerrado|1a17f724-b01b-44c3-b9a1-2a004a9595ee
Executado|c97f34c8-83fd-4893-a379-fc15bb4e5a82
Identificado|e2001078-3039-4ace-af1d-642c03fe0be2
Programado|4d0453c9-49d3-43ec-9dc6-a592cf9d8c56
Vigente|69713866-6767-4724-9f94-facd1a263030


In [12]:
for firm in Firm.objects.filter(company=company).order_by('name'):
    try:
        print("|".join([
            firm.name,
            str(firm.uuid),
        ]))
    except StopIteration:
        continue

Aprovadores ANTT|ff2576e2-db90-4607-b87b-6fb1ab7ac917
Bureau Veritas - Campo|bca7af73-fc40-406b-8ca8-9a0954e41966
Bureau Veritas - Escritório|f569c1f3-eaa4-4494-9296-bc780d48cf43
Bureau Veritas - Programação|3d6ce580-0c1b-41bc-b735-1a6b37c06735
Campo/Escritório|d8e33492-4339-40fb-a3e0-312fa2c8d706
Civil: SGE|c51e2071-67ca-44ff-ad4a-4a005cf95591
Civil: SGOAC|a08c9011-5ebe-463b-b9ab-f267f8adb83d
Civil: SGOAE|3e814127-3500-46c7-921b-60b8e971c6bd
Civil: SGS|8e3f277d-cd02-444f-a49b-c8d724f92c9d
Civil: SGTEC|89bfb142-a735-499b-bde5-05e9225199fc
CLT 01 - RJ|895d1be9-8d20-4203-9f1e-8a4c2c82f0d7
COLETA 01 - 101|f5dd149b-3845-4fbd-b86d-7893d51184a7
COLETA 01 - SP|daee2529-953b-4259-a5e9-efe44ec2b429
COLETA 02 - 101|6a28bc67-6496-46d0-aed0-1c3adb0f310a
COLETA 02 - SP|b001498a-1dd6-43a3-bb80-168a2bf4bf7c
COLETA 03 - SP|8946baf9-3919-4262-ba32-759f404863a7
Conservação|5a54bf97-a41e-435b-b3fe-75081f6aaa90
E - 1|6f96d908-85c9-4eb3-a00b-76f21763ca64
E - 2|434d3e05-e6e3-4d4f-9aac-536326a6a20e
E - 3|b71